<a href="https://colab.research.google.com/github/meng430/repo0/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q pyngrok flask_ngrok # for colab
!pip install -q flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 13.5 MB/s eta 0:00:00


In [ ]:
from re import template
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('ngrok_auth_token'))
    public_url = ngrok.connect(5000)
    print("🌐 Public URL:", public_url)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage, ConfirmTemplate, MessageAction,
    CarouselTemplate,
    CarouselColumn,
    URIAction,
    PostbackAction
)

app = Flask(__name__)

@app.route("/", methods=['GET'])
def index():
  return 'hello!'


configuration = Configuration(access_token=userdata.get('Line_Channel_access_token'))
handler = WebhookHandler(userdata.get('Line_Channel_secret'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
  with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)

    action = event.message.text
    if action == 'confirm':
      reply = TemplateMessage(
        alt_text='這是確認視窗',
        template = ConfirmTemplate(
          text='你喜歡吃薯條嗎?',
          actions=[
            MessageAction(label='Yes', text='薯條好好吃'),
            MessageAction(label='No', text='不喜歡薯條')
          ]
        )
      )
    elif action == 'carousel':
      carousel_template = CarouselTemplate(
        columns=[
          CarouselColumn(
            thumbnailImageurl='https://cdn.pixabay.com/photo/2020/06/23/14/33/french-fries-5332766_1280.jpg',
            title='馬鈴薯薯條',
            text='這是馬鈴薯薯條。',
            actions=[
              URIAction(label='維基百科',uri='https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Truffle_Fries_%28Unsplash%29.jpg/500px-Truffle_Fries_%28Unsplash%29.jpg'),
              URIAction(label='YouTube',uri='https://www.youtube.com/watch?v=57feiF0Gub4&t=220s&ab_channel=%E8%85%A6%E6%B4%9E%E5%A4%A7%E9%96%8B'),
              MessageAction(label='投票', text='我投馬鈴薯薯條一票'),
            ]
          ),

          CarouselColumn(
            thumbnailImageurl='https://cdn.pixabay.com/photo/2016/01/26/00/53/potatoe-1161819_1280.jpg',
            title='地瓜薯條',
            text='這是地瓜薯條。',
            actions=[
                URIAction(label='維基百科',uri='https://zh.wikipedia.org/zh-tw/%E7%94%98%E6%A2%85%E8%96%AF%E6%A2%9D'),
                URIAction(label='YouTube',uri='https://www.youtube.com/watch?v=huJHsYwzzO8&ab_channel=ETtoday%E6%97%85%E9%81%8A%E9%9B%B2'),
                MessageAction(label='投票', text='我投地瓜薯條一票'),
            ]
          )
        ]
      )
      reply = TemplateMessage(
        alt_text='這是輪播視窗',
        template=carousel_template
      )
    else:
      reply = TextMessage(text='Thanks!')

    line_bot_api.reply_message(
      ReplyMessageRequest(
        reply_token=event.reply_token,
        messages=[
          reply
        ]
      )
    )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

🌐 Public URL: NgrokTunnel: "https://d0b2-34-58-9-52.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://d0b2-34-58-9-52.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [14/May/2025 06:33:26] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/May/2025 06:34:42] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/May/2025 06:34:46] "POST /callback HTTP/1.1" 200 -
